In [10]:
%%html

<style>
.jp-MarkdownOutput {
    font-size: 2.5em !important;
}
.jp-MarkdownOutput table {
    font-size: 1em !important;
}
.jp-OutputArea-output pre {
    font-size: 2em !important;
}
.cm-content {
    font-size: 2em !important;
}
.page-id-xx, html {
    scrollbar-width: none; /* FF */
}
::-webkit-scrollbar {
    width: 0px; /* Chrome & Edge */
}
.jp-Notebook.jp-mod-commandMode .jp-Cell.jp-mod-selected {
    background: none;
}
</style>

<img src="img/title.svg" style="width: 90%; border: 1px solid black; margin: 3em auto;">

## Outline

* Who wants high performance and why?
* Understanding why Python is slow
* Python escape hatches:
  * NumPy
  * Awkward Array
  * JIT-compilation
* Special topics:
  * JIT-compilation for GPUs
  * The Python garbage collector
  * The Python GIL (Global Interpreter Lock)
  * Memory-mapping
* No conclusions: we'll stop when we run out of time

## Who wants high performace and why?

* "I'm shepherding a computation that runs for months and 5% faster means 5% less electricity, operating costs, and $CO_2$."

* "I'm building an interactive app, and if each user-initiated action completes in less than human reaction time (100 ms), the app will feel snappy."

* "I'm analyzing a large dataset, and if the computation completes over a lunch break instead of overnight, I'll be able to run it more often to do more in-depth investigations."

* "I'm trying to do &lt;anything at all&gt;, and I can't because I run out of RAM, open file handles, TTYs, ..."

This is me:

* "I'm analyzing a large dataset, and if the computation completes over a lunch break instead of overnight, I'll be able to run it more often to do more in-depth investigations."
* "I'm trying to do &lt;anything at all&gt;, and I can't because I run out of RAM, open file handles, TTYs, ..."

<br><br><br>

Therefore, I usually only care about **speed** when it's an order of magnitude gain and **memory** when necessary.

<img src="img/clock-rate-timeline-1.svg" style="width: 90%; margin: 3em auto;">

<img src="img/clock-rate-timeline-2.svg" style="width: 90%; margin: 3em auto;">

<img src="img/clock-rate-timeline-3.svg" style="width: 90%; margin: 3em auto;">

<img src="img/clock-rate-timeline-4.svg" style="width: 90%; margin: 3em auto;">

Hardware baseline is a given.

There are now only three ways to speed up code:

1. parallel processing

2. fixing bloopers: what you thought it was doing is not what it's doing, or there's a faster method you just didn't know about

3. turning off dynamic features that you don't need

<img src="img/dynamic-features.svg" style="width: 60%; margin: 3em auto;">

"Dynamic feature":

* decisions are made in the loop that scales with dataset size

<br>

"that you don't need":

* information is known before starting the loop that scales with dataset size

Dynamic features are often built into the language

| | dynamic allocation | reference count | garbage collector | runtime evaluation | virtual machine | type reflection | parallel scheduling |
|:--|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| Fortran77 | | | | | | | |
| C | ✓ | | | | | | |
| C++ | ✓ | `shared_ptr<T>` | | | | vtable | stdlib |
| Rust | ✓ | `Rc<T>` | | | | vtable | ✓ |
| Swift | ✓ | ✓ | | | | vtable | ✓ |
| Julia | ✓ | | ✓ | ✓ | | ✓ | stdlib |
| Go | ✓ | | ✓ | | | vtable | ✓ |
| JVM | ✓ | | ✓ | | ✓ | ✓ | stdlib |
| Lua | ✓ | | ✓ | ✓ | ✓ | ✓ | |
| Python | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | stdlib |

<img src="img/benchmark-games-2023.svg" style="width: 80%; margin: 3em auto;">

## Understanding why Python is slow

## Python escape hatches: NumPy

## Python escape hatches: Awkward Array

## Python escape hatches: JIT-compilation

## Special topics: JIT-compilation for GPUs

## Special topics: the Python garbage collector

## Special topics: the Python GIL (Global Interpreter Lock)

## Special topics: memory mapping